In [ ]:
import pandas as pd
from collections import Counter

import os

import shutil
import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
# from sklearn.metrics import classification_report
# import matplotlib.pyplot as plt

In [59]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((512, 512)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])  # grayscale için
    ]),
    'validation': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ]),
    'test': transforms.Compose([
        transforms.Grayscale(num_output_channels=1),
        transforms.Resize((512, 512)),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
    ])
}

In [60]:
data_dir = 'data_model/'

image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'validation', 'test']
}

dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=2)
    for x in ['train', 'validation', 'test']
}

class_names = image_datasets['train'].classes
print("Sınıflar:", class_names)


Sınıflar: ['Atelectasis', 'Cardiomegaly', 'Consolidation', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening', 'Pneumothorax']


In [61]:
dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
    for x in ['train', 'validation', 'test']
}

## CNN modeli

In [ ]:
import torch.nn.functional as F

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 64 * 64, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # (1, 512, 512) -> (16, 256, 256)
        x = self.pool(F.relu(self.conv2(x)))  # (16, 256, 256) -> (32, 128, 128)
        x = self.pool(F.relu(self.conv3(x)))  # (32, 128, 128) -> (64, 64, 64)
        x = x.view(-1, 64 * 64 * 64)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


## Modeli oluşturma

In [ ]:
scratch_model = SimpleCNN(num_classes=len(class_names))
scratch_model = scratch_model.to(device)

## Loss ve optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(scratch_model.parameters(), lr=0.001)

## Eğitim başlatma

In [ ]:
trained_scratch_model = train_model(scratch_model, dataloaders, criterion, optimizer, device, num_epochs=10)

## Transfer Learning

In [62]:
model = models.resnet18(pretrained=True)

# İlk katman RGB (3 kanal) bekliyor. 1 kanala uyarladık.
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)

# Son katmanı sınıf sayısına göre değiştirdik.
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, len(class_names))

model = model.to(device)

c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\keser\miniconda3\envs\ysa_env\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [63]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [65]:
def train_model(model, dataloaders, criterion, optimizer, device, num_epochs=10):
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 20)

        for phase in ['train', 'validation']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print(f'{phase.capitalize()} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
    return model


In [ ]:
trained_model = train_model(model, dataloaders, criterion, optimizer, device, num_epochs=10)


Epoch 1/10
--------------------
Train Loss: 2.2079 Acc: 0.1545
Validation Loss: 2.1598 Acc: 0.1800

Epoch 2/10
--------------------
Train Loss: 2.1649 Acc: 0.1664
Validation Loss: 2.1200 Acc: 0.1822

Epoch 3/10
--------------------
Train Loss: 2.1555 Acc: 0.1716
Validation Loss: 2.1325 Acc: 0.1889

Epoch 4/10
--------------------
